### Algorithm to Approximate Stabilizer Rank
* https://arxiv.org/abs/1711.07848 - On the Geometry of Stabilizer States
* $\underline{\text{First Approach - Approximating an arbitrary quantum state with a single stabilizer state (Section 4.1)}}$
    * Outline of approach:
        * Begin with arbitrary state - 
        $$\ket{\psi} = \sum_{i = 1}^{k} \alpha_i \ket{s_i}$$
        * Start at stabilizer state $\ket{s_m}$ such that it has largest $|\alpha_i|$
        * At each iteration, evaluate $N(\ket{s_m})$ (nearest neighbors of $\ket{s_m}$) by computing 
        $$\max_{\ket{\phi} \in N(\ket{s_m})} |\bra{\phi}\ket{\psi}|$$
* $\underline{\text{Second Approach - }}$

In [16]:
import stim
import numpy as np
from typing import List, Tuple

In [21]:
%run -i NonCliff_1Meas.ipynb 

[(0.7071067811865475-0j)ZX] + [(-0.7071067811865475+0j)ZZ]
The minimum eigenvalue for this pauli sum is: (-0.9999999999999996+0j)
The associated eigenvector is: [ 0.92387953+0.j -0.38268343-0.j -0.        -0.j -0.        -0.j]


In [25]:
np.argmax(np.abs(min_eigvec))

0

In [ ]:
def state_max_coeff(eig_vec: np.array) -> np.array:
    """ 
    Finds the constituent stabilizer state (computational basis state) with 
    largest coefficient amplitude

    Parameters:
    -----------
    eig_vec - Eigenvector 

    Returns:
    --------
    State with largest coefficient amplitude
    """
    ret_vec = np.zeros((len(eig_vec),), dtype=complex)
    max_pos = np.argmax(np.abs(eig_vec))
    ret_vec[max_pos] = 1+0j
    return ret_vec

def find_nearest_neighbors(stab_state: np.array) -> List:
    """ 
    Find and return all nearest neighbor states to chosen stabilizer state 'stab_state'

    Parameters:
    -----------
    stab_state - Stabilizer state for which we wish to find nearest neighbors

    Returns:
    --------
    List of nearest neighbor states
    """

def optimal_nearest_neighbor(eig_vec: np.array) -> np.array:
    """ 
    Find nearest neighbor stabilizer state with maximum inner product

    Parameters:
    -----------
    eig_vec - Eigenvector we are trying to approximate

    Returns:
    --------
    Single stabilizer state that is closest to 'eig_vec'
    """
    